In [ ]:
!sh get_data.sh

In [ ]:
import numpy as np
from scipy.special import factorial
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
from load_data import get_data

In [ ]:
from try_4 import PyIBP
np.random.seed(45222)

def init_ibm(data):
    Xs = len(data)
    return PyIBP(data, (1, 1, 1), [(1.7, 1, 1)] * Xs, [(0.5, 1, 1)] * Xs)

In [ ]:
X_train, X_test = get_data(include_labels=True)

In [ ]:
ibm = init_ibm([X_train[0], X_train[1], X_train[2]])

In [ ]:
ibm.run_sampler(iters=5000)

In [ ]:
def predict(model, i, Xs):
    best = (-float('inf'), None, None)
    for Z in model.Zs:
        m = Z.sum(axis=0) - Z
        lps = np.sum(np.log(np.where(Z == 1, m, model.N - m) / model.N), axis=1)
        for f, X in zip(model.feats[:-1], Xs[:-1]):
            X_bar = Z @ f.weights(Z)
            diffs = np.sum((X - X_bar) ** 2, axis=1) / (2 * f.var_x)
            lps -= diffs + np.log(2 * np.pi) / 2 + np.log(f.var_x) * f.D / 2
        i = np.argmax(lps)
        if lps[i] > best[0]:
            best = lps[i], Z, i
    best_lp, best_Z, best_i = best
    W = model.feats[-1].weights(best_Z)
    X_w_pred = best_Z[best_i] @ W
    print(X_w_pred, Xs[-1])
    print((best_i == i) == (Xs[-1][np.argmax(X_w_pred)] == 1))
    return Xs[-1][np.argmax(X_w_pred)] == 1

In [ ]:
XS = X_train
N = len(XS[0])
sum(predict(ibm, i, [X[i] for X in [XS[0], XS[2]]]) for i in range(N)) / N